## Base model performance benchmarking with GuideLLM

This notebook evaluates the system-level performance of the **base model**. It uses GuideLLM, an open-source benchmarking tool designed to measure the performance of large language models deployed with **vLLM**. The results from this notebook serve as a baseline for comparing performance against the compressed model deployment. 

**Goal**

Establish baseline performance metrics for the base model to compare with the performance of the compressed model under identical serving conditions.

**Key actions**:

- Start a vLLM server to host the compressed model.

- Collect metrics such as token throughput, Time to First Token (TTFT), Inter-Token Latency (ITL), and End-to-end request latency.

- Save results in JSON format.

**Outcome**:

- Performance metrics for the base model.

- A baseline to use for comparison against the compressed model's performance.

For details on system level performance benchmarking and GuideLLM, see [Performance Benchmarking with GuideLLM](../docs/System_Level_Performance_Benchmarking.md)

### Install dependencies

In [ ]:
!pip install -qqU .

In [ ]:
import os

from guidellm.benchmark import GenerativeBenchmarksReport
from utils import generate, stream

### Check GPU memory

To make sure that you have enough GPU memory to run this notebook:

1. In a terminal window, run the `nvidia-smi` command.

2. If there are processes that are using GPU memory that this notebook requires, run the `kill -9 <pid>` command for each process to stop it.



### Launch an inference server (vLLM) for the base model

Set up a vLLM inference server to host your base model and to expose an OpenAI-compatible API endpoint. This server is required so that GuideLLM can benchmark system-level performance, such as throughput, latency, and time-to-first-token. In Module 6, you can compare the performance benchmarks of the base and compressed models.

**Resources used** : 46GB L40S GPU x 1

For more information about using vLLM, see [Launch Inference Servers for the Base and Compressed Models using vLLM](../docs/Model_Serving_vLLM.md).

####  Set up environment variables

In [ ]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

#### Run the vLLM server

This example uses the configuration for a single-node, single-GPU set up to launch a vLLM server for the base model. 

1. Open a terminal window.
   
2. Make sure that the working directory is the `02_Base_Performance_Benchmarking` directory.
   
   `pwd`
   
3. Run the following command to serve the base model by using vLLM:

```bash
vllm serve \
  "../base_model/RedHatAI-Llama-3.1-8B-Instruct" \
  --host 127.0.0.1 \
  --port 8000 \
  --gpu-memory-utilization 0.6 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1 \
  --max-model-len 2048
```

4. Wait for the server to start, as indicated by the following status updates:

    ```INFO:     Started server process [166518]```\
    ```INFO:     Waiting for application startup.```\
    ```INFO:     Application startup complete.```

**NOTE:** You can safely ignore the following warning message:

`The tokenizer you are loading from '../base_model/RedHatAI-Llama-3.1-8B-Instruct' with an incorrect regex pattern... This will lead to incorrect tokenization.`

#### Verify that the vLLM server is accessible

Use the  **generate** helper function (defined in [utils.py](./utils.py)) to simplify sending requests to the locally-served vLLM model.

This function wraps the OpenAI-compatible Chat Completions API exposed by vLLM.

In [ ]:
base_model_path = "../base_model/RedHatAI-Llama-3.1-8B-Instruct"

In [ ]:
# For non streaming results
response = generate(
    model=base_model_path,
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
)
print(response)

In [ ]:
# For streaming results
res = ""
for chunk in stream(
    model=base_model_path,
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
):
    res += chunk
    print(chunk, end="", flush=True)

### Run performance benchmarking

After you verify that the vLLM server for the base model is running, you can proceed with benchmarking the base model's performance by using **GuideLLM**.

1. Identify values for the following parameters:

   - **target**: The URL of the vLLM inference server started in the previous step, for example: `http://127.0.0.1:8000` If the vLLM server is running on a different port, update the `target` accordingly.

   - **output-path**: The path to save the benchmarking results.

2. Open a terminal window.
   
4. Make sure that the working directory is the `02_Base_Performance_Benchmarking` directory.

   `pwd`
   
6. Check the following command and, if needed, edit the **target** and **output-path** parameters. Then run the command in the terminal.

    ```bash
    guidellm benchmark \
      --target "http://127.0.0.1:8000" \
      --profile sweep \
      --max-seconds 120 \
      --data "prompt_tokens=1024,output_tokens=512" \
      --output-path "../results/base_performance_benchmarks.json"
    ```

#### View the results

The result of the `guidellm benchmark` command is formatted in multiple tables. The results display in the terminal and are saved to the directory defined by `output-path`.

1. **Request Latency Statistics (Completed Requests)**

This table focuses on how **long** requests take and the latency characteristics of the server.

```text

ℹ Request Latency Statistics (Completed Requests)
|=============|=========|========|=========|=========|======|======|=======|=======|
| Benchmark   | Request Latency || TTFT             || ITL        || TPOT         ||
| Strategy    | Sec             || ms               || ms         || ms           ||
|             | Mdn     | p95    | Mdn     | p95     | Mdn  | p95  | Mdn   | p95   |
|-------------|---------|--------|---------|---------|------|------|-------|-------|
| synchronous | 11.4    | 11.4   | 115.9   | 124.1   | 22.2 | 22.2 | 22.3  | 22.4  |
| throughput  | 62.3    | 92.4   | 33854.1 | 60812.2 | 55.7 | 92.4 | 121.6 | 180.5 |
| constant    | 12.5    | 12.6   | 130.4   | 143.1   | 24.3 | 24.3 | 24.5  | 24.5  |
| constant    | 13.2    | 13.2   | 133.6   | 144.9   | 25.6 | 25.6 | 25.8  | 25.8  |
| constant    | 14.0    | 14.1   | 133.5   | 144.9   | 27.1 | 27.2 | 27.3  | 27.5  |
| constant    | 14.7    | 14.8   | 138.7   | 151.9   | 28.6 | 28.6 | 28.8  | 28.9  |
| constant    | 16.5    | 16.6   | 140.2   | 156.9   | 32.0 | 32.2 | 32.3  | 32.4  |
| constant    | 17.5    | 17.5   | 143.2   | 157.9   | 33.9 | 34.0 | 34.1  | 34.3  |
| constant    | 18.5    | 18.7   | 146.8   | 161.8   | 36.0 | 36.3 | 36.2  | 36.5  |
| constant    | 20.4    | 20.4   | 147.0   | 162.4   | 39.6 | 39.7 | 39.8  | 39.9  |
|=============|=========|========|=========|=========|======|======|=======|=======|
```

2.  **Server Throughput Statistics**

This table focuses on how many requests a server can handle per second. Throughput can be thought of as the **rate** (or time required) of processing. 
```text
ℹ Server Throughput Statistics
|=============|=====|======|=======|=======|========|========|========|=======|=======|========|
| Benchmark   | Requests                |||| Input Tokens   || Output Tokens || Total Tokens  ||
| Strategy    | Per Sec   || Concurrency  || Per Sec        || Per Sec       || Per Sec       ||
|             | Mdn | Mean | Mdn   | Mean  | Mdn    | Mean   | Mdn    | Mean  | Mdn   | Mean   |
|-------------|-----|------|-------|-------|--------|--------|--------|-------|-------|--------|
| synchronous | 0.1 | 0.1  | 1.0   | 1.0   | 92.5   | 101.8  | 45.1   | 44.8  | 45.1  | 137.4  |
| throughput  | 0.5 | 1.7  | 125.0 | 100.5 | 132.2  | 3094.4 | 604.5  | 885.0 | 607.5 | 2713.7 |
| constant    | 0.3 | 0.2  | 3.0   | 3.2   | 303.9  | 314.3  | 138.2  | 135.9 | 138.3 | 416.8  |
| constant    | 0.5 | 0.4  | 6.0   | 5.7   | 519.6  | 530.3  | 204.8  | 227.9 | 205.2 | 698.9  |
| constant    | 0.7 | 0.6  | 10.0  | 8.6   | 735.7  | 746.1  | 262.5  | 318.9 | 262.7 | 977.8  |
| constant    | 0.9 | 0.8  | 13.0  | 11.5  | 951.1  | 962.4  | 344.7  | 408.1 | 344.8 | 1251.4 |
| constant    | 1.1 | 0.9  | 18.0  | 15.5  | 1169.7 | 1178.3 | 422.8  | 491.6 | 423.4 | 1507.6 |
| constant    | 1.3 | 1.1  | 23.0  | 19.3  | 1383.0 | 1394.3 | 464.6  | 576.5 | 465.1 | 1767.8 |
| constant    | 1.5 | 1.3  | 28.0  | 23.4  | 1598.5 | 1610.6 | 497.3  | 658.7 | 498.2 | 2019.9 |
| constant    | 1.7 | 1.4  | 34.0  | 28.5  | 1827.6 | 1826.5 | 576.5  | 734.0 | 577.6 | 2250.6 |
|=============|=====|======|=======|=======|========|========|========|=======|=======|========|

```
#### Base Model Performance Summary
1. Max concurrency under load: 34.0 (Concurrency Mdn)
2. Max output tokens per second under load: 576.5 (Output tokens per sec Mdn)
3. Request latency under load: 20.4 (Request Latency in secs Mdn)
4. Time to first token under load: 147.0 (TTFT ms Mdn)
5. Inter token latency under load: 39.6 (ITL ms Mdn)

#### SLO Analysis

Assume the Service Level Objective (SLO) is:

    TTFT ≤ 200 milliseconds for 95% of requests (p95) with optimal concurrency

Given the SLO of TTFT ≤ 200 ms for 95% of requests (p95) at optimal concurrency, the base model meets this requirement. At the highest tested concurrency of 34 requests, the p95 TTFT is 162.4 ms, which satisfies the SLO.

These results establish the performance baseline for the base model. After you compress the model, you can benchmark it under the same conditions to determine whether model compression leads to improved TTFT while continuing to meet the SLO.

In [ ]:
# Run this cell after the benchmarking process in the terminal completes
report = GenerativeBenchmarksReport.load_file(
    path="../results/base_performance_benchmarks.json",
)
base_benchmarks = report.benchmarks

In [ ]:
base_benchmarks[0]